https://mccormickml.com/2019/07/22/BERT-fine-tuning/

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r"D:\Data\wine-reviews\winemag-data-130k-v2.csv")

In [3]:
designation = df['designation']

In [4]:
len(designation.value_counts())

37979

In [5]:
vc = designation.value_counts()

In [6]:
vc.mean()

2.435714473788146

In [7]:
vc.head()

Reserve         2009
Estate          1322
Reserva         1259
Riserva          698
Estate Grown     621
Name: designation, dtype: int64

In [8]:
vc.loc['Reserve']

2009

In [9]:
kept_wines = vc[vc>3]

In [10]:
kept_wines.head()

Reserve         2009
Estate          1322
Reserva         1259
Riserva          698
Estate Grown     621
Name: designation, dtype: int64

In [11]:
kept_wines[:20]

Reserve            2009
Estate             1322
Reserva            1259
Riserva             698
Estate Grown        621
Brut                513
Dry                 413
Barrel sample       375
Crianza             343
Estate Bottled      342
Vieilles Vignes     308
Brut Rosé           276
Gran Reserva        261
Barrel Sample       252
Tradition           238
Old Vine            221
Extra Dry           204
Rosé of             172
Rosé                166
Réserve             153
Name: designation, dtype: int64

In [12]:
kept_wines[-20:]

Abbaye de Morgeot Premier Cru      4
Steinklotz Grand Cru               4
Five Mountain                      4
Anna-Christina                     4
Messias Selection                  4
Cuvée Don Reca Limited Release     4
Pentad                             4
Silice                             4
Ramal Vineyard                     4
Single Vineyard Temis              4
Millantu                           4
Manchester Ridge                   4
Steinriegl                         4
Silver                             4
Alter Ego de Château Palmer        4
Rocche di Castelletto              4
Williams Selyem Estate Vineyard    4
Yates Conwill Vineyard             4
1105                               4
Colombelle L'Original              4
Name: designation, dtype: int64

fig, ax = plt.subplots()
vc.plot(ax=ax, kind='bar')

In [13]:
vc.quantile(0.6)

1.0

In [14]:
df.shape

(129971, 14)

If I'm only going to use wines with more than one review, that's less than 40% of the wines

In [15]:
df.columns

Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'region_1', 'region_2', 'taster_name',
       'taster_twitter_handle', 'title', 'variety', 'winery'],
      dtype='object')

In [16]:
countries = df['country']

In [17]:
print(countries.value_counts())

US                        54504
France                    22093
Italy                     19540
Spain                      6645
Portugal                   5691
Chile                      4472
Argentina                  3800
Austria                    3345
Australia                  2329
Germany                    2165
New Zealand                1419
South Africa               1401
Israel                      505
Greece                      466
Canada                      257
Hungary                     146
Bulgaria                    141
Romania                     120
Uruguay                     109
Turkey                       90
Slovenia                     87
Georgia                      86
England                      74
Croatia                      73
Mexico                       70
Moldova                      59
Brazil                       52
Lebanon                      35
Morocco                      28
Peru                         16
Ukraine                      14
Czech Re

In [18]:
df['taster_name'].describe()

count         103727
unique            19
top       Roger Voss
freq           25514
Name: taster_name, dtype: object

In [19]:
df

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,NaN,Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
129969,129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


In [20]:
wine_counts = kept_wines.values.tolist()